In [ ]:
#| default_exp signature_validators

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from pymoq.argument_validators import ArgumentValidator, ArgumentFunctionValidator, argument_validator_from_argument 
from typing import Any
from fastcore.basics import patch_to
from itertools import chain

In [ ]:
#| hide
from fastcore.test import test_fail

# Signature validators
> Checking if signatures are valid

A signature validator is simply a collection of argument validators. Its `is_valid` methods checks for a given list of arguments if all argument validators return valid.

In [ ]:
#| export
class SignatureValidator:
    "This class holds a list of argument validators and can evaluate a list of arguments against those validators"
    def __init__(self, argument_validators: list[ArgumentValidator]):
        self.argument_validators = argument_validators
        self._named_validators = {validator.name: validator
                                  for validator in self.argument_validators}
        
        self._positional_validators = {validator.position: validator
                                      for validator in self.argument_validators}
        
        names = [validator.name for validator in self.argument_validators]
        if len(names) != len(set(names)):
            raise ValueError(f"List of argument validators contains duplicate names: {names}")
            
        positions = [validator.position for validator in self.argument_validators]
        if len(positions) != len(set(positions)):
            raise ValueError(f"List of argument validators contains duplicate positions: {positions}")
        
    def is_valid(self, *args: list[Any], **kwargs: dict[str, Any]) -> bool:
        if len(args) > len(self.argument_validators): return False
    
        # positional arguments
        for position, value in enumerate(args):
            if not position in self._positional_validators.keys(): return False
            
            if not self._positional_validators[position].is_valid(value): return False
        
        # named arguments
        for name,value in kwargs.items():
            if name not in self._named_validators: return False
        
            if not self._named_validators[name].is_valid(value): return False
        
        return True
    
    def __str__(self) -> str:
        validator_string = '\n\t'.join(map(str, self._positional_validators.values())) + '\n\t'.join(map(str, self._named_validators.values()))
        return 'SignatureValidator:\n\t' + \
    'Positional:\n\t\t' + '\n\t\t'.join(map(str, self._positional_validators.values())) + \
    '\n\tNamed\n\t\t' + '\n\t\t'.join(map(str, self._named_validators.values()))
    
    def __repr__(self): return str(self)

In [ ]:
any_int = argument_validator_from_argument(int, name="firstArgument", position=0)
second_any_int = argument_validator_from_argument(int, name="secondArgument", position=1)

s = SignatureValidator([any_int, second_any_int])
s

SignatureValidator:
	Positional:
		ArgumentFunctionValidator(argument_name:firstArgument, position=0): any_int
		ArgumentFunctionValidator(argument_name:secondArgument, position=1): any_int
	Named
		ArgumentFunctionValidator(argument_name:firstArgument, position=0): any_int
		ArgumentFunctionValidator(argument_name:secondArgument, position=1): any_int

Calling a signature validator with only positional arguments works as expected:

In [ ]:
assert s.is_valid(1,1)
assert not s.is_valid(1,"1")
assert not s.is_valid("1", 1)

Named arguments:

In [ ]:
assert s.is_valid(1, secondArgument=1)
assert not s.is_valid(1, named="1")

#### Edge cases

Argument validators contain more than one element with the same name

In [ ]:
test_fail(lambda : SignatureValidator([any_int, any_int]), "duplicate names")

Argument validators contain more than one element with the same position

In [ ]:
second_any_int._position = any_int._position
test_fail(lambda : SignatureValidator([any_int, second_any_int]), "duplicate positions")
second_any_int._position = any_int._position + 1

More arguments than validators:

In [ ]:
assert not s.is_valid(1,1,1)

## Ease of use: Construct from arguments

In [ ]:
#| export

VERBOSE = False

def signature_validator_from_arguments(argument_names: list[str], *args, **kwargs) -> SignatureValidator:
    "Construct a `SignatureValidator` by smartly constructing `ArgumentValidators` when no actual argument validators are given"
    argument_validators = []
    
    # positional arguments
    if VERBOSE: print('Positional:')
    for position, (name,argument) in enumerate(zip(argument_names,args)):
        arg_validator = argument_validator_from_argument(argument, name, position, verbose=VERBOSE)
        argument_validators.append(arg_validator)
        
    if VERBOSE: print('\nNamed:')
    # keyword arguments
    for name,named_argument in kwargs.items():
        if isinstance(named_argument, ArgumentValidator):
            position = named_argument.position + 1
        else:
            last_position = max(map(lambda val: val.position, argument_validators))
            position = last_position + 1

        arg_validator = argument_validator_from_argument(named_argument, name, position, verbose=VERBOSE)
        argument_validators.append(arg_validator)
    
    return SignatureValidator(argument_validators)

If there are actual ArgumentValidators given:

In [ ]:
sign_val = signature_validator_from_arguments(['a','b'], ArgumentFunctionValidator(lambda a: isinstance(a, int), name='a', position=0), b=ArgumentFunctionValidator(lambda b: isinstance(b, str), name='b', position=1))

assert sign_val.is_valid(1, '1')
assert not sign_val.is_valid("1")
assert not sign_val.is_valid(1,1)

If there is a callable given, a `ArgumentFunctinValidator` is constructed:

In [ ]:
sign_val = signature_validator_from_arguments(['a', 'b'], lambda a: isinstance(a, int), b=lambda b: isinstance(b, str))

assert sign_val.is_valid(1, '1')
assert not sign_val.is_valid("1")
assert not sign_val.is_valid(1,1)

# Build library

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()